# Data collection

In [ ]:
import requests
import pandas as pd
from apscheduler.schedulers.background import BackgroundScheduler

## define carpark_data scheduler function.

- call API to retrieve data at the timepoint that runs the code.

In [2]:
def carpark_data():

  # Step 1: get data from API 
  response = requests.get('https://api.data.gov.sg/v1/transport/carpark-availability').json()

  # Step 2: create dataframe 
  df= pd.DataFrame(columns=['total_lot','lot_type','lots_available','carpark_number','update_date','update_time'])

  # Step 3: extract data from response of API. for each elements, store it into specific variable.
  for i in range(len(response['items'][0]['carpark_data'])):
    total_lot = response['items'][0]['carpark_data'][i]['carpark_info'][0]['total_lots']
    lot_type = response['items'][0]['carpark_data'][i]['carpark_info'][0]['lot_type']
    lots_available = response['items'][0]['carpark_data'][i]['carpark_info'][0]['lots_available']
    carpark_number = response['items'][0]['carpark_data'][i]['carpark_number']
    update_datetime = response['items'][0]['carpark_data'][i]['update_datetime']
    update_date = str(update_datetime).split('T')[0]
    update_time = str(update_datetime).split('T')[1]

    # Step 4: for each elements, append variables to the dataframe.
    df = df.append(
        {'total_lot':total_lot,'lot_type':lot_type,
        'lots_available':lots_available,'carpark_number':carpark_number,
        'update_date':update_date,'update_time':update_time
        },ignore_index=True)
    # Step 5: save dataframe into csv file.
  return df.to_csv('SG_carpark_availability.csv',mode='a',index=True, header=False)

In [ ]:
# initialize background scheduler which runs the code in background.
sched = BackgroundScheduler(daemon=True)

# load function defined earlier to collect data. and set to run the code hourly.
sched.add_job(carpark_data, 'interval', hours= 1)

# start schedular
sched.start()

In [ ]:
#sched.shutdown(wait=False)